In [2]:
import numpy as np
import scanpy as sc
import anndata as ad
import scipy.io
import gzip
import pooch
# Data retrieval
import pandas as pd
import os
import scipy.sparse as sp
from anndata import AnnData
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import rc_context
import seaborn as sns
from scipy.stats import gaussian_kde
from scipy.stats import ttest_ind
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
import glob
import scvi
from tqdm import tqdm
import anndata as ad
import tempfile
import torch
from rich import print
from scib_metrics.benchmark import Benchmarker



/home/pyxie/miniconda3/envs/custom_jupyter_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pyxie/miniconda3/envs/custom_jupyter_env/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/pyxie/miniconda3/envs/custom_jupyter_env/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [ ]:
# Load the CSV metadata file
metadata_file = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/GSE156728_metadata.txt"  # Change to your actual file name
metadata_df = pd.read_csv(metadata_file, sep="\t", index_col=0)
from IPython.display import display
# Display the dataframe in Jupyter Notebook
display(metadata_df)



In [4]:
# setup
figpath = '/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/figures'

In [ ]:
# lets set the default figure settings
sc.settings.set_figure_params(dpi_save=300, dpi=100)
sc.settings.figdir = figpath

# Define the path where your datasets are stored
data_path = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/"
# Create output directory if it doesn't exist
#output_dir = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/loaded_data"
#os.makedirs(output_dir, exist_ok=True)

### UNCOMMENT THIS SECTION IF LOADING RAW FILES
## Loading Raw Files
# Get a list of all dataset files (adjust the pattern based on your filenames)
# file_list = glob.glob(data_path + "*.txt")  # Reads all .txt files
# file_list
# # Load datasets with a progress bar
# dataframes = []
# for file in tqdm(file_list, desc="Loading Datasets", unit="file"):
#     df = pd.read_csv(file, sep="\t", index_col=0)
#     dataframes.append(df)
#     df
#     del df

### UNCOMMENT THIS SECTION IF LOADING ALREADY CONCATENATED DATA
# Define the path where your datasets are stored
data_path = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/concatenated_data_v3.csv"
df = pd.read_csv(data_path, sep=",", index_col=0)
concatenated_df = df
concatenated_df


# # t_cell_integrated_path = "data/GSE99254_NSCLC.TCell.S11769.norm.centered.txt"
# t_cell_integrated_path = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/GSE156728_BC_10X.CD8.counts.txt"


# # Read as a pandas DataFrame
# tcelldf = pd.read_csv(t_cell_integrated_path, sep="\t", index_col=0)  # Assuming first column is genes
# # tcelldf = pd.read_csv(t_cell_integrated_path, sep="\t", index_col=0)  # Assuming first column is genes

# # Display first few rows
# print(tcelldf.head())
# #print(tcelldf2.head())
# gene_symbols = tcelldf["symbol"].values

In [6]:
#concatenated_df = pd.concat(dataframes, axis=1, join='inner')
#concatenated_df.fillna(0, inplace=True)
#data_path = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/concatenated_data_v3.csv"
#concatenated_df.to_csv(data_path, index=True)
#concatenated_df

In [7]:
i = 0
batch_labels = ['adata_ov', 'adata_ucec', 'adata_paca', 'adata_bc', 'adata_thca', 'adata_mm', 'adata_rc', 'adata_chol', 'adata_ftc', 'adata_esca', 'adata_bcl']
# Define DataFrame shapes (corresponding to batch assignments)
dataframe_shapes = [(24148, 3517), (24148, 19926), (24148, 5957), (24148, 4291), (24148, 33450),
                    (28855, 8629), (24148, 16544), (12582, 300), (24148, 767), (24148, 12526), (28855, 3482)]

    

In [8]:
tcelladata = ad.AnnData(X=concatenated_df.transpose())
tcelladata.obs['samples'] = concatenated_df.columns
tcelladata.layers["raw_count"] = concatenated_df.transpose()
# Create an empty batch column
tcelladata.obs['batch'] = ""
# Assign batch labels based on cell indices
start_idx = 0
cell_counts = [shape[1] for shape in dataframe_shapes]
print(cell_counts)
for i, num_cells in enumerate(cell_counts):
    end_idx = start_idx + num_cells  # End index for this batch
    tcelladata.obs.iloc[start_idx:end_idx, tcelladata.obs.columns.get_loc('batch')] = batch_labels[i]
    start_idx = end_idx  # Move to next batch

print(tcelladata.obs.columns)


[3517, 19926, 5957, 4291, 33450, 8629, 16544, 300, 767, 12526, 3482]

Index(['samples', 'batch'], dtype='object')

In [ ]:
tcelladata.obs['no_genes'] = (tcelladata.X > 0).sum(axis=1)
tcelladata.obs['total_counts'] = tcelladata.X.sum(axis=1)
# mito_genes = tcelladata.var.index.str.startswith("MT")
# tcelladata.obs['percent_mito'] = np.sum(tcelladata[:, mito_genes].X, axis=1) / np.sum(tcelladata.X, axis=1)
# ribosomal genes
tcelladata.var["percent_mito"] = tcelladata.var.index.str.startswith(("MT"))
# hemoglobin genes
#tcelladata.var["hb"] = tcelladata.var.index.str.contains("^HB[^(P)]")
# Filter out genes that are expressed in very few cells
print("# cells, # genes before filtering:", tcelladata.shape)
sc.pp.filter_genes(tcelladata, min_counts=100)
sc.pp.filter_cells(tcelladata, min_counts=10)
print("# cells, # genes after filtering:", tcelladata.shape)
sc.pp.calculate_qc_metrics(tcelladata, qc_vars=["percent_mito"], inplace=True)
tcelladata  = tcelladata[tcelladata.obs['pct_counts_percent_mito']  < 2]
min_genes = 100
min_counts = 10
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
tcelladata.obs['no_genes'].hist(bins=60, ax=axes[0], color='skyblue')
axes[0].axvline(min_genes, color='red', linestyle='dashed')
tcelladata.obs['total_counts'].hist(bins=60, ax=axes[1], color='skyblue')
axes[1].axvline(min_counts, color='red', linestyle='dashed')
tcelladata.obs['pct_counts_percent_mito'].hist(bins=60, ax=axes[1], color='skyblue')
axes[1].axvline(min_counts, color='red', linestyle='dashed')
for ax, metric in zip(axes, ['no_genes', 'total_counts']):
    ax.set_title(metric)
    plt.tight_layout()
    plt.show()

In [10]:
tcelladata.var_names_make_unique()
print(tcelladata.obs.columns)
tcelladata

Index(['samples', 'batch', 'no_genes', 'total_counts', 'n_counts',
       'n_genes_by_counts', 'log1p_n_genes_by_counts', 'log1p_total_counts',
       'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes',
       'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes',
       'total_counts_percent_mito', 'log1p_total_counts_percent_mito',
       'pct_counts_percent_mito'],
      dtype='object')

AnnData object with n_obs × n_vars = 108758 × 11561
    obs: 'samples', 'batch', 'no_genes', 'total_counts', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_percent_mito', 'log1p_total_counts_percent_mito', 'pct_counts_percent_mito'
    var: 'percent_mito', 'n_counts', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    layers: 'raw_count'

In [ ]:
sc.pl.violin(
    tcelladata,
    ["no_genes", "total_counts", "pct_counts_percent_mito"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
sns.jointplot(
    data=tcelladata.obs,
    x="log1p_total_counts",
    y="log1p_n_genes_by_counts",
    kind="hex",
)

In [13]:
sc.pp.normalize_total(tcelladata) # counts per million normalizes for the sequencing depth, natural log normalization is used to adjust the data distribution to be semi-normal
# Sum expression counts for each cell (column-wise sum)
cell_sums = np.array(tcelladata.X.sum(axis=1)).flatten()

print(cell_sums[:20])
sc.pp.log1p(tcelladata)
tcelladata.raw = tcelladata

[3886.0002 3886.     3886.     3885.9998 3886.     3886.     3886.
 3886.0002 3886.     3886.     3886.     3886.     3886.     3886.
 3886.     3886.     3886.     3886.0002 3885.9998 3886.    ]

In [14]:
# identify variable genes
# identify variable genes
sc.pp.highly_variable_genes(tcelladata)
tcelladata

tcelladata.var.loc[tcelladata.var.highly_variable].sort_values("dispersions", ascending=False)
tcelladata.var.rename(columns={'features': 'gene_symbols'}, inplace=True)

In [ ]:
# plot variablility distribution
sc.pl.highly_variable_genes(tcelladata)

In [16]:
# scvi.model.SCVI.setup_anndata(tcelladata, layer = "raw_count", batch_key="batch")

In [17]:
# model = scvi.model.SCVI(tcelladata, n_layers=2, n_latent=30, gene_likelihood="nb")

In [18]:
# model.train()

In [19]:
# SCVI_LATENT_KEY = "X_scVI"
# tcelladata.obsm[SCVI_LATENT_KEY] = model.get_latent_representation()

In [20]:
# sc.pp.neighbors(tcelladata, use_rep=SCVI_LATENT_KEY)
# sc.tl.leiden(tcelladata)

In [21]:
sc.pp.scale(tcelladata, max_value=10)

In [ ]:
# PCA, default subsets to highly variable features
sc.tl.pca(tcelladata)

# A visualization that is useful for determining how many PCs to include
sc.pl.pca_variance_ratio(tcelladata, n_pcs=20)

In [ ]:
# print(tcelldf.index)
print(tcelladata.var.index)
print("CD3D" in tcelladata.var.index)  # Should return True if CD4 is present

tcelladata

In [ ]:
sc.pp.neighbors(tcelladata, n_pcs=15)
# Perform Leiden clustering
sc.tl.leiden(tcelladata, resolution=0.1, flavor="igraph", n_iterations=2)  # Adjust resolution for more or fewer clusters
sc.tl.umap(tcelladata)
# plot UMAP
print(tcelladata.obs.columns)  # Ensure 'batch' exists

fig = sc.pl.umap(tcelladata, color=['leiden'], legend_loc="on data")
sc.pl.umap(tcelladata, color="leiden", title="Pan-cancer T-cell UMAP",
           legend_fontsize=6, legend_loc="on data")

In [ ]:
# Would be interesting to plot the 21 different tumor types on the overall CD8+ UMAP
# Should lower the clustering resolution
sc.pl.umap(tcelladata, color="batch", title="Pan-cancer T-cell UMAP",
           legend_fontsize=6, legend_loc="on data")

In [ ]:
# label clusters based on the metadata
print(metadata_df.index)
filtered_metadata = metadata_df[metadata_df.index.isin(tcelladata.obs_names)]

# Ensure the number of cells matches
print(f"Filtered metadata: {filtered_metadata.shape[0]} cells")
print(f"tcelladata: {tcelladata.shape[0]} cells")

# Set Cell_ID as index in metadata for easy merging
# filtered_metadata = filtered_metadata.set_index("cellID")

# Assign clusters to tcelladata.obs
tcelladata.obs["meta_cluster"] = filtered_metadata.loc[tcelladata.obs_names, "meta.cluster"].values

# Assign clusters to tcelladata.obs
tcelladata.obs["sample_location"] = filtered_metadata.loc[tcelladata.obs_names, "loc"].values

tcelladata.obs["cancerType"] = filtered_metadata.loc[tcelladata.obs_names, "cancerType"].values
# Verify assignment
print(tcelladata.obs["meta_cluster"].head())

print(tcelladata.obs["meta_cluster"].value_counts())

print(tcelladata.obs["sample_location"].head())

print(tcelladata.obs["sample_location"].value_counts())

print(tcelladata.obs["cancerType"].head())

print(tcelladata.obs["cancerType"].value_counts())

In [ ]:
# Plot UMAP colored by meta_cluster
sc.pl.umap(tcelladata, color="meta_cluster", title="UMAP of T-cell Data by Meta Cluster",legend_fontsize=10, palette="tab10", frameon=False, save = "Pan_cancer_UMAP_plot.png")
# plt.gcf().savefig('/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/figures/umap_cell_type_highres.png', dpi=600)

In [ ]:
# plot common marker genes for pbmc
print(tcelladata.var.head())  # Check if 'genes' column exists
print(tcelladata.var.index[:10])  # Check current index
print("CD8A" in tcelladata.var.index)
print("CD4" in tcelladata.var.index)
print(tcelladata.obs.columns)
print(tcelladata.var.columns)

genes_to_plot = ['CD3D','CD4','CCR7','FOXP3', 'IL2RA','CD3D','CD8A', 'PIEZO1', 'SELL', 'PDCD1', 'CTLA4', 'CD3G', 'JCHAIN','RNF114']
sc.pl.umap(tcelladata, color=genes_to_plot, use_raw=True,
                sort_order=True, ncols=3)

naive_markers = ['SELL', 'LEF1', 'CCR7']
inhibitory_receptors = ['LAG3', 'TIGIT', 'PDCD1', 'HAVCR2', 'CTLA4']
cytokine_genes = ['IL2', 'GZMA', 'GNLY', 'PRF1', 'GZMB', 'GZMK', 'IFNG', 'NKG7']
co_stim_molecules = ['CD28', 'TNFRSF14', 'ICOS', 'TNFRSF14', 'ICOS', 'TNFRSF9']
transcription_factors = ['EOMES', 'HOPX', 'TBX21', 'ZEB2', 'ZNF683', 'HIF1A', 'ID2', "TOX"]
tregmarkers = ['IL2RA', 'FOXP3', 'IKZF2']
sc.pl.umap(tcelladata, color=naive_markers, use_raw=True,
                sort_order=True, ncols=3, cmap = "viridis")
sc.pl.umap(tcelladata, color=inhibitory_receptors, use_raw=True,
                sort_order=True, ncols=3, cmap = "viridis")
sc.pl.umap(tcelladata, color=cytokine_genes, use_raw=True,
                sort_order=True, ncols=3, cmap = "viridis")
sc.pl.umap(tcelladata, color=transcription_factors, use_raw=True,
                sort_order=True, ncols=3, cmap = "viridis")
sc.pl.umap(tcelladata, color=tregmarkers, use_raw=True,
                sort_order=True, ncols=3, cmap = "viridis")

In [ ]:
new_cluster_names = ['CD8+ Naive/Central Memory T-cells',
                    'CD8+ Progenitor Exhausted T cells',
                    'CD8+ Cytotoxic T-cells',
                    'CD8+ Memory T-cells',
                    'CD8+ Terminal Exhausted T-cells']
tcelladata.rename_categories('leiden', new_cluster_names)

sc.pl.umap(tcelladata, color="leiden", title="Pan-cancer T-cell UMAP",
           legend_fontsize=6, legend_loc="on data", save = "labelled_UMAP.png")

# is there an automatic t-cell labelling software?

In [33]:
sc.tl.embedding_density(tcelladata, basis='umap', groupby="batch")

In [ ]:
sc.tl.rank_genes_groups(tcelladata, groupby="meta_cluster", method="wilcoxon")
sc.pl.rank_genes_groups(tcelladata, n_genes=10, sharey=False)

In [ ]:
sc.tl.rank_genes_groups(tcelladata, groupby="cancerType", method="wilcoxon")
sc.pl.rank_genes_groups(tcelladata, n_genes=10, sharey=False)

In [106]:
groups = tcelladata.uns["rank_genes_groups"]["names"].dtype.names 
top_genes_df_1 = sc.get.rank_genes_groups_df(tcelladata, group = groups[0]).head(10)
top_genes_df_2 = sc.get.rank_genes_groups_df(tcelladata, group = groups[1]).head(10)
top_genes_df_3 = sc.get.rank_genes_groups_df(tcelladata, group = groups[2]).head(10)
top_genes_df_4 = sc.get.rank_genes_groups_df(tcelladata, group = groups[3]).head(10)
top_genes_df_5 = sc.get.rank_genes_groups_df(tcelladata, group = groups[4]).head(10)
top_genes_df_6 = sc.get.rank_genes_groups_df(tcelladata, group = groups[5]).head(10)
top_genes_df_7 = sc.get.rank_genes_groups_df(tcelladata, group = groups[6]).head(10)
top_genes_df_8 = sc.get.rank_genes_groups_df(tcelladata, group = groups[7]).head(10)
top_genes_df_9 = sc.get.rank_genes_groups_df(tcelladata, group = groups[8]).head(10)
top_genes_df_10 = sc.get.rank_genes_groups_df(tcelladata, group = groups[9]).head(10)
# # Combine into one DataFrame
df_all = pd.concat([top_genes_df_1, top_genes_df_2, top_genes_df_3,top_genes_df_4, top_genes_df_5, top_genes_df_6,top_genes_df_7, top_genes_df_8, top_genes_df_9,top_genes_df_10], keys=groups)
print(df_all.head())
print(df_all.shape)


/local/scratch/pyxie/slrmtmp.47757444/ipykernel_9648/403439408.py:13: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  df_all = pd.concat([top_genes_df_1, top_genes_df_2, top_genes_df_3,top_genes_df_4, top_genes_df_5, top_genes_df_6,top_genes_df_7, top_genes_df_8, top_genes_df_9,top_genes_df_10], keys=groups)


names     scores  logfoldchanges  pvals  pvals_adj
BC 0   RPS26  72.348297             NaN    0.0        0.0
   1   RPS29  58.132893             NaN    0.0        0.0
   2  MALAT1  50.201145             NaN    0.0        0.0
   3    LMNA  45.020432             NaN    0.0        0.0
   4   YPEL5  40.095505             NaN    0.0        0.0

(100, 5)

In [ ]:
# Extract top genes as a list
top_genes_df = pd.DataFrame(df_all, columns=["names", "scores", "logfoldchanges", "pvals", "pvals_adj"])  # Adjust columns
top_genes_list = top_genes_df["names"]  # Convert column to a list
print(top_genes_list[:10])  # Verify the output
df_unique = top_genes_list.drop_duplicates()
print(df_unique)
geo_gene_list_mechanotransduction = [
    "ABHD12", "ACTA1", "ADGRV1", "ANGPT2", "ANKRD1", "ANKRD23", "ANO3", "AQP1", "ASIC2", "ASIC3",
    "ATAT1", "ATOH7", "ATP1A2", "ATP8A2", "ATR", "BACE1", "BAD", "BAG3", "BAK1", "BCL10",
    "BDKRB1", "BGLAP", "BMP6", "BNIP3", "BTG2", "CALB1", "CAPN2", "CASP1", "CASP2", "CASP5",
    "CASP8", "CASP8AP2", "CAV3", "CD40", "CDH2", "CHEK1", "CHI3L1", "CHRNA10", "CHRNA9", "CITED2",
    "CLCN6", "CNN2", "CNTNAP2", "COL11A1", "COL1A1", "COL6A1", "CRADD", "CSRP3", "CTNNB1", "CXCL10",
    "CXCL12", "CXCR4", "DAG1", "DCANP1", "DDR2", "DMD", "DRD2", "EDN1", "EGFR", "ENG",
    "ETV1", "F11R", "FADD", "FAS", "FGF2", "FOS", "FOSB", "FOSL1", "FYN", "GADD45A",
    "GAP43", "GATA4", "GCLC", "GDF5", "GPI", "GSN", "HABP4", "HPN", "HTR2A", "HTT",
    "IGF1R", "IGFBP2", "IHH", "IL13", "IL1B", "IL33", "IRF1", "ITGA2", "ITGAM", "ITGB3",
    "JUP", "KCNA1", "KCNA5", "KCNC1", "KCNJ2", "KCNK2", "KCNK4", "KCNQ1", "KCNQ3", "KIAA0319",
    "KIT", "KRT5", "LARGE1", "LHFPL5", "LRP11", "LTBR", "MAG", "MAP1B", "MAP2K4", "MAP3K1",
    "MAP3K14", "MAP3K2", "MAPK14", "MAPK3", "MAPK8", "MBD2", "MDK", "MEIS2", "MKKS", "MMP14",
    "MMP2", "MPO", "MYD88", "NEUROG1", "NFKB1", "NFKBIA", "NPPA", "NRXN1", "NRXN2",
    "NTRK1", "P2RX3", "P2RX7", "P2RY1", "PDE2A", "PDZD7", "PHF24", "PIEZO1", "PIEZO2", "PIK3CA",
    "PJVK", "PKD1", "PKD1L1", "PKD1L2", "PKD1L3", "PKD2", "PKD2L1", "PKD2L2", "PKDREJ", "PLEC",
    "POSTN", "PPL", "PSPH", "PTCH1", "PTGER4", "PTGS2", "PTK2", "PTK2B", "PTN", "RAF1",
    "RELA", "RETN", "RPS6KB1", "RYR2", "SCEL", "SCN11A", "SCN1A", "SCN9A", "SCX", "SERPINE2",
    "SHANK3", "SLC1A3", "SLC26A5", "SLC2A1", "SLC38A2", "SLC8A1", "SLC9A1", "SLITRK6", "SOST", "SOX9",
    "SRC", "STAT1", "STRA6", "STRBP", "STRC", "SUN1", "TACR1", "TCAP", "TGFB1", "THBS1",
    "TIFAB", "TLR3", "TLR4", "TLR5", "TLR7", "TLR8", "TMC1", "TMC2", "TMEM120A", "TMEM150C",
    "TMEM87A", "TNC", "TNF", "TNFRSF10A", "TNFRSF10B", "TNFRSF11A", "TNFRSF1A", "TNFRSF8", "TNFSF14", "TRPA1",
    "TRPV4", "TTN", "TUBA1A", "TXNIP", "UCN", "USP53", "WHRN", "WNT11", "XPA", "XPC", "VCAM1"]

exhaustion_genes = [
    "PDCD1", "LAG3", "TIGIT", "CTLA4", "HAVCR2", "TOX", 
]
mech_genes_in_data = [x for x in geo_gene_list_mechanotransduction if x in tcelladata.var_names]
print(len(mech_genes_in_data))

cytokine_genes = ['IL2', 'GZMA', 'GNLY', 'PRF1', 'GZMB', 'GZMK', 'IFNG','TNF', 'NKG7']
# Find overlapping genes
overlapping_genes = set(df_unique).intersection(set(geo_gene_list_mechanotransduction))
overlapping_genes = list(overlapping_genes)
print("Overlapping Genes:", overlapping_genes)

sc.pl.heatmap(
    tcelladata, 
    var_names=mech_genes_in_data,   # List of top marker genes
    groupby="cancerType", # Group by cancer types
    cmap="magma",        # Change color scale if needed
    swap_axes=True,        # Display genes on Y-axis
    show=True,
    show_gene_labels = True,
    vmin=-2.5, vmax=2.5,  # Adjust for better contrast
)

In [ ]:
batch_mapping = {
   "PTC": "Blood",
   "NTC": "Adjacent",
   "TTC": "Tumor",
   "PTH": "Blood",
   "NTH": "Adjacent",
   "TTH": "Tumor",
   "PTR": "Blood",
   "NTR": "Adjacent",
   "TTR": "Tumor",
   "PTY": "Blood",
   "NTY": "Adjacent",
   "TTY": "Tumor"   
}
batch_mapping2 = {
   "PTC": "Blood",
   "NTC": "Solid",
   "TTC": "Solid",
   "PTH": "Blood",
   "NTH": "Solid",
   "TTH": "Solid",
   "PTR": "Blood",
   "NTR": "Solid",
   "TTR": "Solid",
   "PTY": "Blood",
   "NTY": "Solid",
   "TTY": "Solid"   
}

# batch_mapping = {
#    "PTC": "Blood CD8+",
#    "NTC": "Adjacent CD8+",
#    "TTC": "Tumor CD8+",
#    "PTH": "CD4+",
#    "NTH": "CD4+",
#    "TTH": "CD4+",
#    "PTR": "CD4+",
#    "NTR": "CD4+",
#    "TTR": "CD4+",
#    "PTY": "CD4+",
#    "NTY": "CD4+",
#    "TTY": "CD4+" 
# }

# Create a new batch column by mapping sample prefixes
tcelladata_cd8.obs["batch"] = tcelladata_cd8.obs["samples"].str[:3].map(batch_mapping)

# Create a new batch column by mapping sample prefixes
tcelladata_cd8.obs["mechanics"] = tcelladata_cd8.obs["samples"].str[:3].map(batch_mapping2)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    tcelladata, groupby="leiden", standard_scale="var", n_genes=10
)

In [ ]:
tcelladata

In [ ]:
# Scoring each batch based on the expression of the exhaustion gene set
exhaustion_genes = [
    "PDCD1", "LAG3", "TIGIT", "CTLA4", "HAVCR2", "TOX"
]

exhaustion_genes_in_data = [x for x in exhaustion_genes if x in tcelladata.var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(tcelladata, gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = tcelladata.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": tcelladata.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Exhaustion_Score": tcelladata.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/exhaustion_scores_by_cancer_type.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()




In [ ]:
# Scoring each batch based on the expression of the exhaustion gene set
exhaustion_genes = [
    "PDCD1", "LAG3", "TIGIT", "CTLA4", "HAVCR2", "TOX"
]

exhaustion_genes_in_data = [x for x in exhaustion_genes if x in tcelladata.var_names]
print(len(exhaustion_genes_in_data))

sub_ex_tcelladata = tcelladata[tcelladata.obs["meta_cluster"].isin(["CD8.c11.Tex.PDCD1", "CD8.c12.Tex.CXCL13"])].copy() 

# Compute exhaustion scores
sc.tl.score_genes(sub_ex_tcelladata, gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = sub_ex_tcelladata.obs["batch"]  # Ensure this column exists
clusters = sub_ex_tcelladata.obs["meta_cluster"]  # Cluster labels
print(cancer_types)



# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": sub_ex_tcelladata.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Cluster": clusters.values,
    "Exhaustion_Score": sub_ex_tcelladata.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/exhaustion_scores_by_cancer_type_Tex_only.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()




In [ ]:
# Scoring each batch based on the expression of the exhaustion gene set
Viscoelasticity_gene_set = [
    "FOSB", "FOS", "DUSP1", "TMEM67", "SHC4", "CTLA4", "CCR4"
]

viscoelasticity_genes_in_data = [x for x in Viscoelasticity_gene_set if x in tcelladata.var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(tcelladata, gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = tcelladata.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": tcelladata.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Exhaustion_Score": tcelladata.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/exhaustion_scores_by_cancer_type.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()




In [ ]:
# Scoring each batch based on the expression of the exhaustion gene set
Mechanical_response_to_stimulus = [
    "ABHD12", "ACTA1", "ADGRV1", "ANGPT2", "ANKRD1", "ANKRD23", "ANO3", "AQP1", "ASIC2", "ASIC3",
    "ATAT1", "ATOH7", "ATP1A2", "ATP8A2", "ATR", "BACE1", "BAD", "BAG3", "BAK1", "BCL10",
    "BDKRB1", "BGLAP", "BMP6", "BNIP3", "BTG2", "CALB1", "CAPN2", "CASP1", "CASP2", "CASP5",
    "CASP8", "CASP8AP2", "CAV3", "CD40", "CDH2", "CHEK1", "CHI3L1", "CHRNA10", "CHRNA9", "CITED2",
    "CLCN6", "CNN2", "CNTNAP2", "COL11A1", "COL1A1", "COL6A1", "CRADD", "CSRP3", "CTNNB1", "CXCL10",
    "CXCL12", "CXCR4", "DAG1", "DCANP1", "DDR2", "DMD", "DRD2", "EDN1", "EGFR", "ENG",
    "ETV1", "F11R", "FADD", "FAS", "FGF2", "FOS", "FOSB", "FOSL1", "FYN", "GADD45A",
    "GAP43", "GATA4", "GCLC", "GDF5", "GPI", "GSN", "HABP4", "HPN", "HTR2A", "HTT",
    "IGF1R", "IGFBP2", "IHH", "IL13", "IL1B", "IL33", "IRF1", "ITGA2", "ITGAM", "ITGB3",
    "JUP", "KCNA1", "KCNA5", "KCNC1", "KCNJ2", "KCNK2", "KCNK4", "KCNQ1", "KCNQ3", "KIAA0319",
    "KIT", "KRT5", "LARGE1", "LHFPL5", "LRP11", "LTBR", "MAG", "MAP1B", "MAP2K4", "MAP3K1",
    "MAP3K14", "MAP3K2", "MAPK14", "MAPK3", "MAPK8", "MBD2", "MDK", "MEIS2", "MKKS", "MMP14",
    "MMP2", "MPO", "MTPN", "MYD88", "NEUROG1", "NFKB1", "NFKBIA", "NPPA", "NRXN1", "NRXN2",
    "NTRK1", "P2RX3", "P2RX7", "P2RY1", "PDE2A", "PDZD7", "PHF24", "PIEZO1", "PIEZO2", "PIK3CA",
    "PJVK", "PKD1", "PKD1L1", "PKD1L2", "PKD1L3", "PKD2", "PKD2L1", "PKD2L2", "PKDREJ", "PLEC",
    "POSTN", "PPL", "PSPH", "PTCH1", "PTGER4", "PTGS2", "PTK2", "PTK2B", "PTN", "RAF1",
    "RELA", "RETN", "RPS6KB1", "RYR2", "SCEL", "SCN11A", "SCN1A", "SCN9A", "SCX", "SERPINE2",
    "SHANK3", "SLC1A3", "SLC26A5", "SLC2A1", "SLC38A2", "SLC8A1", "SLC9A1", "SLITRK6", "SOST", "SOX9",
    "SRC", "STAT1", "STRA6", "STRBP", "STRC", "SUN1", "TACR1", "TCAP", "TGFB1", "THBS1",
    "TIFAB", "TLR3", "TLR4", "TLR5", "TLR7", "TLR8", "TMC1", "TMC2", "TMEM120A", "TMEM150C",
    "TMEM87A", "TNC", "TNF", "TNFRSF10A", "TNFRSF10B", "TNFRSF11A", "TNFRSF1A", "TNFRSF8", "TNFSF14", "TRPA1",
    "TRPV4", "TTN", "TUBA1A", "TXNIP", "UCN", "USP53", "WHRN", "WNT11", "XPA", "XPC"
]


viscoelasticity_genes_in_data = [x for x in Mechanical_response_to_stimulus if x in tcelladata.var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(tcelladata, gene_list=viscoelasticity_genes_in_data , score_name="Mechanics_Score")

# Extract cancer-type labels
cancer_types = tcelladata.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": tcelladata.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Mechanics_Score": tcelladata.obs["Mechanics_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/mechanics_scores_by_cancer_type.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Mechanics_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Mechanics Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()




In [ ]:
# Subset tcelladata for only t-cells in the tumor itself
# 1. Exhaustion Score
# 2. Mechanosensation Score
tcelladata_tumor = tcelladata[tcelladata.obs["sample_location"] == 'T'].copy() 

exhaustion_genes = [
    "PDCD1", "LAG3", "TIGIT", "CTLA4", "HAVCR2", "TOX"
]

sub_tcelladata_tumor = tcelladata_tumor[tcelladata_tumor.obs["meta_cluster"].isin(["CD8.c11.Tex.PDCD1", "CD8.c12.Tex.CXCL13"])].copy() 

exhaustion_genes_in_data = [x for x in exhaustion_genes if x in sub_tcelladata_tumor.var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(sub_tcelladata_tumor, gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = sub_tcelladata_tumor.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": sub_tcelladata_tumor.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Exhaustion_Score": sub_tcelladata_tumor.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/correlation_file/C1_exhaustion_scores_for_tumor_and_exhaustedonly.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()



In [ ]:
# Subset tcelladata for only t-cells in the adjacent tissue itself
# 1. Exhaustion Score
# 2. Mechanosensation Score
tcelladata_normal_tissue = tcelladata[tcelladata.obs["sample_location"] == 'N'].copy() 


exhaustion_genes = [
    "PDCD1", "LAG3", "TIGIT", "CTLA4", "HAVCR2", "TOX"
]

exhaustion_genes_in_data = [x for x in exhaustion_genes if x in tcelladata_normal_tissue.var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(tcelladata_normal_tissue, gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = tcelladata_normal_tissue.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": tcelladata_normal_tissue.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Exhaustion_Score": tcelladata_normal_tissue.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/exhaustion_scores_for_adjacent_tissue_only.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()


In [ ]:
# Subset tcelladata for only t-cells in the blood itself
# 1. Exhaustion Score
# 2. Mechanosensation Score
tcelladata_blood = tcelladata[tcelladata.obs["sample_location"] == 'P'].copy() 


exhaustion_genes = [
    "PDCD1", "LAG3", "TIGIT", "CTLA4", "HAVCR2", "TOX"
]

exhaustion_genes_in_data = [x for x in exhaustion_genes if x in tcelladata_blood .var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(tcelladata_blood , gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = tcelladata_blood.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": tcelladata_blood.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Exhaustion_Score": tcelladata_blood.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/exhaustion_scores_for_blood_only.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()

In [54]:
expr_df = tcelladata_tumor.raw.to_adata().to_df()
# Group by cancer type and compute the mean expression per gene
mean_expr_by_cancer = expr_df.groupby(tcelladata_tumor.obs["batch"]).mean()

# Define the output file path
output_csv_path = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/mean_expr_by_cancer_solid_tumor_only"
# Save the DataFrame as a CSV file
mean_expr_by_cancer.to_csv(output_csv_path)



/local/scratch/pyxie/slrmtmp.47757444/ipykernel_9648/2787000557.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_expr_by_cancer = expr_df.groupby(tcelladata_tumor.obs["batch"]).mean()


In [ ]:
geo_gene_list = [
    "ABHD12", "ACTA1", "ADGRV1", "ANGPT2", "ANKRD1", "ANKRD23", "ANO3", "AQP1", "ASIC2", "ASIC3",
    "ATAT1", "ATOH7", "ATP1A2", "ATP8A2", "ATR", "BACE1", "BAD", "BAG3", "BAK1", "BCL10",
    "BDKRB1", "BGLAP", "BMP6", "BNIP3", "BTG2", "CALB1", "CAPN2", "CASP1", "CASP2", "CASP5",
    "CASP8", "CASP8AP2", "CAV3", "CD40", "CDH2", "CHEK1", "CHI3L1", "CHRNA10", "CHRNA9", "CITED2",
    "CLCN6", "CNN2", "CNTNAP2", "COL11A1", "COL1A1", "COL6A1", "CRADD", "CSRP3", "CTNNB1", "CXCL10",
    "CXCL12", "CXCR4", "DAG1", "DCANP1", "DDR2", "DMD", "DRD2", "EDN1", "EGFR", "ENG",
    "ETV1", "F11R", "FADD", "FAS", "FGF2", "FOS", "FOSB", "FOSL1", "FYN", "GADD45A",
    "GAP43", "GATA4", "GCLC", "GDF5", "GPI", "GSN", "HABP4", "HPN", "HTR2A", "HTT",
    "IGF1R", "IGFBP2", "IHH", "IL13", "IL1B", "IL33", "IRF1", "ITGA2", "ITGAM", "ITGB3",
    "JUP", "KCNA1", "KCNA5", "KCNC1", "KCNJ2", "KCNK2", "KCNK4", "KCNQ1", "KCNQ3", "KIAA0319",
    "KIT", "KRT5", "LARGE1", "LHFPL5", "LRP11", "LTBR", "MAG", "MAP1B", "MAP2K4", "MAP3K1",
    "MAP3K14", "MAP3K2", "MAPK14", "MAPK3", "MAPK8", "MBD2", "MDK", "MEIS2", "MKKS", "MMP14",
    "MMP2", "MPO", "MTPN", "MYD88", "NEUROG1", "NFKB1", "NFKBIA", "NPPA", "NRXN1", "NRXN2",
    "NTRK1", "P2RX3", "P2RX7", "P2RY1", "PDE2A", "PDZD7", "PHF24", "PIEZO1", "PIEZO2", "PIK3CA",
    "PJVK", "PKD1", "PKD1L1", "PKD1L2", "PKD1L3", "PKD2", "PKD2L1", "PKD2L2", "PKDREJ", "PLEC",
    "POSTN", "PPL", "PSPH", "PTCH1", "PTGER4", "PTGS2", "PTK2", "PTK2B", "PTN", "RAF1",
    "RELA", "RETN", "RPS6KB1", "RYR2", "SCD4","SCEL", "SCN11A", "SCN1A", "SCN9A", "SCX", "SERPINE2",
    "SHANK3", "SLC1A3", "SLC26A5", "SLC2A1", "SLC38A2", "SLC8A1", "SLC9A1", "SLITRK6", "SOST", "SOX9",
    "SRC", "STAT1", "STRA6", "STRBP", "STRC", "SUN1", "TACR1", "TCAP", "TGFB1", "THBS1",
    "TIFAB", "TLR3", "TLR4", "TLR5", "TLR7", "TLR8", "TMC1", "TMC2", "TMEM120A", "TMEM150C",
    "TMEM87A", "TNC", "TNF", "TNFRSF10A", "TNFRSF10B", "TNFRSF11A", "TNFRSF1A", "TNFRSF8", "TNFSF14", "TRPA1",
    "TRPV4", "TTN", "TUBA1A", "TXNIP", "UCN", "USP53", "WHRN", "WNT11", "XPA", "XPC"
]

sub_tcelladata_tumor = tcelladata_tumor[tcelladata_tumor.obs["meta_cluster"].isin(["CD8.c11.Tex.PDCD1", "CD8.c12.Tex.CXCL13"])].copy() 

exhaustion_genes_in_data = [x for x in geo_gene_list if x in sub_tcelladata_tumor.var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(sub_tcelladata_tumor, gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = sub_tcelladata_tumor.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": sub_tcelladata_tumor.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Exhaustion_Score": sub_tcelladata_tumor.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/correlation_file/C2_GOmechanotransduction_scores_for_tumor_and_exhaustedonly.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()




In [ ]:
geo_gene_list = [
    "FOSB", "FOS", "GADD45A", "XPC", "IL13", "FAS", "MAPK8", "TNFRSF10B", "STAT1", "SLC2A1",
    "MBD2", "STRBP", "PTGER4", "FYN", "CXCR4", "CTNNB1", "TNFRSF8", "CITED2", "MAP3K2", "TMEM87A",
    "NFKB1", "HTT", "TNFRSF11A", "BAD", "PTK2B", "CASP8", "NFKBIA", "RPS6KB1", "ABHD12", "TUBA1A",
    "TXNIP", "MTPN", "TNFSF14", "CHEK1", "RAF1", "BAK1", "PSPH", "RELA", "HABP4", "ITGA2",
    "CNN2", "GPI", "PTGS2", "BCL10", "SUN1", "BAG3", "BTG2", "CASP2", "MAP2K4", "IRF1",
    "MAPK14", "FADD", "XPA", "CASP8AP2", "SLC9A1", "TNFRSF1A", "BNIP3", "MAP3K14", "MAPK3", "FOSL1",
    "TTN", "PIEZO1", "GCLC", "ATAT1", "TMEM120A", "ATR", "PIK3CA", "GSN", "WHRN", "MYD88",
    "MAP3K1", "PKD1", "TGFB1", "DAG1", "CAPN2", "TNFRSF10A", "ITGAM"
]

# sub_tcelladata_tumor = tcelladata_tumor[tcelladata_tumor.obs["meta_cluster"].isin(["CD8.c11.Tex.PDCD1", "CD8.c12.Tex.CXCL13"])].copy() 
sub_tcelladata_tumor = tcelladata_tumor
exhaustion_genes_in_data = [x for x in geo_gene_list if x in sub_tcelladata_tumor.var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(sub_tcelladata_tumor, gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = sub_tcelladata_tumor.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": sub_tcelladata_tumor.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Exhaustion_Score": sub_tcelladata_tumor.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/correlation_file/C4_overlapping_mechanotransduction_scores_for_tumor_and_all_tcells.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()

In [65]:
# Load your single-cell dataset (modify path accordingly)

# Ensure necessary columns exist in metadata
if "meta_cluster" not in tcelladata_tumor.obs or "batch" not in tcelladata_tumor.obs:
    raise KeyError("Missing required columns 'meta_cluster' or 'cancer_type' in tcelldata.obs. Check your metadata.")

# Count total cells per tumor type
total_cells_per_tumor = tcelladata_tumor.obs["batch"].value_counts()

# Count exhausted T-cells per tumor type
exhausted_cells_per_tumor = (
    tcelladata_tumor.obs[tcelladata_tumor.obs["meta_cluster"].isin(["CD8.c11.Tex.PDCD1", "CD8.c12.Tex.CXCL13", "CD8.c13.Tex.myl12a","CD8.c14.Tex.TCF7"])]["batch"].value_counts()
)

# Compute the proportion of exhausted T-cells per tumor type
proportion_exhausted = (exhausted_cells_per_tumor/total_cells_per_tumor).reset_index()
proportion_exhausted.columns = ["Cancer_Type", "Proportion_Exhausted"]

# Save as CSV
output_csv_path = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/correlation_file/Proportion_exhausted_tcells_by_tumor_type.csv"
proportion_exhausted.to_csv(output_csv_path, index=False)

print(f"CSV file saved at: {output_csv_path}")


CSV file saved at: 
/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/correlation_file/Proportion_exhaust
ed_tcells_by_tumor_type.csv

In [ ]:
geo_gene_list = [
    "FOSB", "FOS", "GADD45A", "XPC", "IL13", "FAS", "MAPK8", "TNFRSF10B", "STAT1", "SLC2A1",
    "MBD2", "STRBP", "PTGER4", "FYN", "CXCR4", "CTNNB1", "TNFRSF8", "CITED2", "MAP3K2", "TMEM87A",
    "NFKB1", "HTT", "TNFRSF11A", "BAD", "PTK2B", "CASP8", "NFKBIA", "RPS6KB1", "ABHD12", "TUBA1A",
    "TXNIP", "MTPN", "TNFSF14", "CHEK1", "RAF1", "BAK1", "PSPH", "RELA", "HABP4", "ITGA2",
    "CNN2", "GPI", "PTGS2", "BCL10", "SUN1", "BAG3", "BTG2", "CASP2", "MAP2K4", "IRF1",
    "MAPK14", "FADD", "XPA", "CASP8AP2", "SLC9A1", "TNFRSF1A", "BNIP3", "MAP3K14", "MAPK3", "FOSL1",
    "TTN", "PIEZO1", "GCLC", "ATAT1", "TMEM120A", "ATR", "PIK3CA", "GSN", "WHRN", "MYD88",
    "MAP3K1", "PKD1", "TGFB1", "DAG1", "CAPN2", "TNFRSF10A", "ITGAM"
]

sub_tcelladata_tumor = tcelladata_[tcelladata_blood.obs["meta_cluster"].isin(["CD8.c11.Tex.PDCD1", "CD8.c12.Tex.CXCL13"])].copy() 

exhaustion_genes_in_data = [x for x in geo_gene_list if x in sub_tcelladata_tumor.var_names]
print(len(exhaustion_genes_in_data))

# Compute exhaustion scores
sc.tl.score_genes(sub_tcelladata_tumor, gene_list=exhaustion_genes_in_data, score_name="Exhaustion_Score")

# Extract cancer-type labels
cancer_types = sub_tcelladata_tumor.obs["batch"]  # Ensure this column exists
print(cancer_types)

# Create a DataFrame with scores and cancer-type labels
df_scores = pd.DataFrame({
    "Cell_ID": sub_tcelladata_tumor.obs["samples"],
    "Cancer_Type": cancer_types.values,
    "Exhaustion_Score": sub_tcelladata_tumor.obs["Exhaustion_Score"]
})

# Group by cancer type and compute mean exhaustion scores
# df_grouped = df_scores.groupby("Cancer_Type")

# Export to CSV
csv_filename = "/labs/delitto/peter/pan_cancer_tcell_atlas_raw_data/Raw_dataset/Raw_dataset/CD8/correlation_file/C3_overlapping_mechanotransduction_scores_for_tumor_and_exhaustedonly.csv"
df_scores.to_csv(csv_filename, index=False)

# --- Generate Box Plot ---
plt.figure(figsize=(12, 6))
sns.boxplot(x="Cancer_Type", y="Exhaustion_Score", data=df_scores)

# Enhance plot aesthetics
plt.xticks(rotation=45, ha="right")
plt.xlabel("Cancer Type")
plt.ylabel("Exhaustion Score")
plt.title("T-cell Exhaustion Score Across Cancer Types")

# Show plot
plt.show()